# How to find the 2D Basis of a Plane using GBpy!

Follow the installation manual to receive the byxtal package and import all the required packages that we need for completing this tutorial.

In [8]:
import byxtal as bxt
import numpy as np
from sympy.matrices import Matrix, eye, zeros;

## Miller Indices and Common Conventions:

Miller Indices are often used to refer to a given crystallograohic plane in crystals. However various conventions are commonly used in determining the Miller Indices that can completely change the plane of reference. For this reason we would like to declare our conventions in defining the Miller Indices to avoid any confusions. By definition Miller Indices are defined as the reciprocal of intercepts of a crystallographic plane with the Unit Cell Basis Vectors. The confusion is often caused by the definition of the Unit Cell. In f.c.c and b.c.c lattices primitive basis vectors are non-orthogonal while the supercell basis vectors are orthogonal. Most importantly since the reciprocal and direct lattices are the identical in f.c.c and b.c.c, the Miller Indices (defined in supercell unit cell) are the same as the plane normal indices. This unique property and convenience of using an orthogonal set of basis vectors is the root cause of the bipartisan approach to defining the Miller Indices. The downfall of such an approach is the fact that it does not have much utility in other types of lattices e.g. hcp. Therefore in GBpy whenever we use the term Miller Indices, we are referring to the reciprocals of intercepts of a crystallographic plane with the primitive Basis Vectors.

Miller Indices: reciprocal of intercepts of a crystallographic plane with the primitive Basis Vectors

or alternatively

Miller Indices: normal vector indices of the plane defined in primitive reciprocal lattice

The above interchangeable definitions are consistently used in GBpy for referring to Miller Indices. Other conventions in defining the Miller Indices are ultimately converted to the above definition.

In the present tutorial we manually extract various Indices and demonstrate how they can be fed to the GBpy. In practice such conversions are not necessary as long as the user is consistent with the definitions of GBpy, and can be used for verifying the answer.

## Problem Definition:

In this tutorial we would like to find the planar basis of a plane with $[2 3 1]$ normal vector defined in supercell f.c.c basis .

Since the normal vector to the plane is provided in the orthogonal f.c.c basis (supercell basis), first we need to obtain the Miller Indices of the plane to be able to use the byxtal package.

## Finding Miller Indices:

A vector in the space can be expressed in any basis, of course with varying components. Vector $\vec{v}$ in basis A can be expressed as:

$$ 
\vec{v} = \mathcal{B}_A v_A
$$

Similarly we can define the plane normal $\vec{n}$ in any basis. For instance we can define $\vec{n}$ in supercell f.c.c basis ($\mathcal{B}_{po}$) or in primitive f.c.c basis $\left( \mathcal{B}_p \right) $; we can write:

We can express the right hand side of the (2) as:

where:

$\Lambda_p^{po}$ is a 3x3 matrix with its columns being the components of basis vectors of po in p basis.

In [10]:
l_p_po = 1.0 * Matrix([[0.,0.5,0.5],[0.5,0.,0.5],[0.5,0.5,0.]])
l_p_po

Matrix([
[  0, 0.5, 0.5],
[0.5,   0, 0.5],
[0.5, 0.5,   0]])

From (6) we need to find the reciprocal of the $\Lambda_{p}^{po}$. To do so use the `bxt.find_csl_dsc.reciprocal_mat()`. For the sake of convenience we abbreviate the imported function `bxt.find_csl_dsc` as `fcd`:

In [7]:
import byxtal.find_csl_dsc as fcd
l_rp_po = fcd.reciprocal_mat(l_p_po)
l_rp_po

Matrix([
[-1.0,  1.0,  1.0],
[ 1.0, -1.0,  1.0],
[ 1.0,  1.0, -1.0]])

In [9]:
l_po_rp = (l_rp_po).inv()
l_po_rp

Matrix([
[  0, 0.5, 0.5],
[0.5,   0, 0.5],
[0.5, 0.5,   0]])

In [11]:
n_po = Matrix([[2], [3], [1]])
n_rp = l_po_rp*n_po
n_rp

Matrix([
[2.0],
[1.5],
[2.5]])

In [12]:
import byxtal.integer_manipulations as iman
ni_rp = iman.int_finder(n_rp)
ni_rp

Matrix([
[4],
[3],
[5]])

## Finding the Planar Basis:

From the previous section we found the Miller Indices of an f.c.c plane with the normal along $[2 3 1]$ to be $(4 3 5)$. Now all we have to do is to pass the obtained indices to `bp_basis`, which is a function that gets the Miller Indices as the input and returns a $3 \times 2$ matrix, where each column is one of the two planar basis vectors. Note that the output of the `bp_basis` is always expressed in the primitive lattice basis. Also the obtained vectors are in the reduced form, meaning that the length of the obtained basis vectors are as close as possible. You can find the bp_basis function in the following path: `byxtal.bp_basis.bp_basis()`.

To find the basis vector of a plane with the Miller Indices of $(4 3 5)$ use the following syntax:

In [16]:
import byxtal.bp_basis as bpb
l_2D_p = Matrix(bpb.bp_basis(ni_rp))
l_2D_p

Matrix([
[ 2.0,  1.0],
[-1.0,  2.0],
[-1.0, -2.0]])

To express the obtained basis in the orthogonal basis (i.e. supercell f.c.c) one needs to perform the following conversion of bases:

$$
\Lambda_{2D}^{po} = \Lambda_{p}^{po} \times \Lambda_{2D}^{p}
$$

In [17]:
l_2D_po = l_p_po*l_2D_p
l_2D_po

Matrix([
[-1.0,    0],
[ 0.5, -0.5],
[ 0.5,  1.5]])

At the interface of a bicrystal, the $\Lambda_{2D}^{po}$ provides a basis for the interface. If the two crystals are related to each other by a $\Sigma$-rotation, the obtained $\Lambda_{2D}^{po}$ is the two-dimensional basis for the two-dimensional coincidence site lattice at the interface. Therefore, the bicrystal conserves its periodicity in the obtained 2D-basis. In other words the obtained basis is in fact the basis for the unit cell of the bicrystal and since it is in the reduced form, it is going to have the smallest skewness, hence ideal for constructing a periodic simulation box.

The above process is frquently repeated for simulation of grain boundaries. Therefore, we have developed a set of functions that make the conversion of indices more convenient and will accept various conventions for the Miller Indices. Please refer to the grain boundary 2D-CSL tutorial for how to use these functions.